In [138]:
import sympy as sp
import mainframe as mf
import nuclei_func as nf
from mainframe import Region
from IPython.display import display, Latex

In [139]:
################################################################################
# Cross section data
################################################################################
micro_abs_water = nf.mixture([0.33, 4.2],[2, 1]) # XSEC DATA ?
micro_abs_U235 = 101 # XSEC DATA ?
micro_fiss_U235 = 678 # XSEC DATA ?

density_water = 1
density_U235 = density_water # NOT SURE

macro_abs_water = nf.macro(micro_abs_water, density_water, 18)
macro_abs_U235 = nf.macro(micro_abs_U235, density_U235, 235)
macro_fiss_U235 = nf.macro(micro_fiss_U235, density_U235, 235)

qual = sp.symbols('qual', positive=True, real=True, lower_than=1)

macro_abs_refl = nf.mixture([macro_abs_water, 0], [1, 0])
macro_fiss = nf.mixture([macro_fiss_U235, 0], [qual, 1-qual])
macro_abs_core = nf.mixture([macro_abs_water, macro_abs_U235], [1-qual, qual])

display(Latex(r'$\Sigma_{f} = ' + sp.latex(macro_fiss) + r'$'))
display(Latex(r'$\Sigma_{a, core} = ' + sp.latex(macro_abs_core) + r'$'))
display(Latex(r'$\Sigma_{a, refl} = ' + sp.latex(macro_abs_refl) + r'$'))

The sum of the qualites values is not 1, at line 4, default behaviour is ignoring.


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [140]:
############################################################################################################
# INPUT EXAMPLE
############################################################################################################
Reflector_SX = Region(0.16, macro_abs_refl, 0, 0, 60, -sp.oo, 0, {'H': 2, 'O': 1})
Core = Region(0.16, macro_abs_core, macro_fiss, 2.44, 60, 0, 50, {'H': 2, 'O': 1, 'U-235': 1}, 25*50)
Reflector_DX = Region(0.16, macro_abs_refl, 0, 0, 60, 50, sp.oo, {'H': 2, 'O': 1})
regions = [Reflector_SX, Core, Reflector_DX]

In [141]:
# Solving the problem
# Capture output to hide debug prints
from IPython.utils.io import capture_output

with capture_output() as captured:
    equations, fluxes, boundaries, interfaces, powers, system = mf.main(regions)


In [142]:
equations, fluxes, boundaries, interfaces, powers, system = mf.main(regions)

The region 1 spanning from -oo to 0 is a boundary region
The regions 1 and 2 are adjacent at 0
The region 3 spanning from 50 to oo is a boundary region
The regions 2 and 3 are adjacent at 50
Diffusion equations:
Eq(-A*phi_1(x)/D + Derivative(phi_1(x), (x, 2)), 0)
Eq(Bg_2**2*phi_2(x) + Derivative(phi_2(x), (x, 2)), 0)
Eq(-A*phi_3(x)/D + Derivative(phi_3(x), (x, 2)), 0)

Fluxes:
Eq(phi_1(x), C1*exp(-1.00807365802306*x) + C2*exp(1.00807365802306*x))
Eq(phi_2(x), C3*sin(5.08862648671295*x*sqrt(qual - 0.0392449142425948)) + C4*cos(5.08862648671295*x*sqrt(qual - 0.0392449142425948)))
Eq(phi_3(x - 50), 7.76321030915348e+21*C5*exp(-1.00807365802306*x) + 1.28812689618999e-22*C6*exp(1.00807365802306*x))

Boundary conditions:
Eq(C1, 0)
Eq(C6, 0)

Interface conditions:
Eq(C1 + C2, C4)
Eq(0.16129178528369*C1 - 0.16129178528369*C2, -0.814180237874072*C3*sqrt(qual - 0.0392449142425948))
Eq(C3*sin(254.431324335648*sqrt(qual - 0.0392449142425948)) + C4*cos(254.431324335648*sqrt(qual - 0.039244914242594

In [143]:
# Print the equations
print('Diffusion equations:')
for eq in equations:
    display(Latex("$" + sp.latex(eq) + "$"))

Diffusion equations:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [144]:
# Print the fluxes
print('\nFluxes:')
for flux in fluxes:
    display(Latex("$" + sp.latex(flux) + "$"))


Fluxes:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [145]:
# Print the boundary conditions
print('\nBoundary conditions:')
for boundary in boundaries:
    display(Latex("$" + sp.latex(boundary) + "$"))


Boundary conditions:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [146]:
# Print the interface conditions
print('\nInterface conditions:')
for interface in interfaces:
    display(Latex("$" + sp.latex(interface) + "$"))


Interface conditions:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [147]:
# Print the power conditions
print('\nPower conditions:')
for power in powers:
    display(Latex("$" + sp.latex(power) + "$"))


Power conditions:


<IPython.core.display.Latex object>

In [148]:
# Print the simplified system in latex
print('\nSimplified system:')
for eq in system:
    display(Latex("$" + sp.latex(eq) + "$"))


Simplified system:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [149]:
# Solve the system
# Get all the unkowns
all_symbols = set()
for eq in system:
    all_symbols.update(eq.free_symbols)

x = sp.symbols('x')
all_symbols.discard(x)

unkowns = [flux.lhs for flux in fluxes]
for i in all_symbols:
    unkowns.append(i)

display(unkowns)
display(system)

[phi_1(x), phi_2(x), phi_3(x - 50), C5, C2, C3, C4, qual]

[Eq(phi_1(x), C2*exp(1.00807365802306*x)),
 Eq(phi_2(x), C3*sin(5.08862648671295*x*sqrt(qual - 0.0392449142425948)) + C4*cos(5.08862648671295*x*sqrt(qual - 0.0392449142425948))),
 Eq(phi_3(x - 50), 7.76321030915348e+21*C5*exp(-1.00807365802306*x)),
 Eq(C2, C4),
 Eq(-0.16129178528369*C2, -0.814180237874072*C3*sqrt(qual - 0.0392449142425948)),
 Eq(C3*sin(254.431324335648*sqrt(qual - 0.0392449142425948)) + C4*cos(254.431324335648*sqrt(qual - 0.0392449142425948)), C5),
 Eq(-0.814180237874072*C3*sqrt(qual - 0.0392449142425948)*cos(254.431324335648*sqrt(qual - 0.0392449142425948)) + 0.814180237874072*C4*sqrt(qual - 0.0392449142425948)*sin(254.431324335648*sqrt(qual - 0.0392449142425948)), 0.16129178528369*C5),
 Eq(4.23928299574468*qual*(-0.196516683354757*C3*cos(254.431324335648*sqrt(qual - 0.0392449142425948))/sqrt(qual - 0.0392449142425948) + 0.196516683354757*C3/sqrt(qual - 0.0392449142425948) + 0.196516683354757*C4*sin(254.431324335648*sqrt(qual - 0.0392449142425948))/sqrt(qual - 0.03924